In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType


In [0]:
df = spark.table("brz.sales")
display(df)


In [0]:
df = df.withColumnRenamed("Unit Cost", "Unit_Cost") \
       .withColumnRenamed("Unit Price", "Unit_Price") \
       .withColumnRenamed("Email Name", "Email_Name")

display(df)


In [0]:
df = df.withColumn(
    "email",
    F.regexp_extract(F.col("Email_Name"), r"\(([^)]+)\)", 1)
)

display(df)


In [0]:
df = df.withColumn(
    "last_name",
    F.trim(
        F.split(
            F.regexp_extract(F.col("Email_Name"), r"\):\s*(.*)", 1),
            ","
        )[0]
    )
)

display(df)


In [0]:
df = df.withColumn(
    "first_name",
    F.trim(
        F.split(
            F.regexp_extract(F.col("Email_Name"), r"\):\s*(.*)", 1),
            ","
        )[1]
    )
)

display(df)


In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS slv")

df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("slv.sales")
